In [170]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import anthropic
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [171]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAh


In [172]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
MODEL = 'ollama3.2'
openai = OpenAI()

client = anthropic.Anthropic()

In [173]:
# websie class
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [179]:
plans = Website("https://www.t-mobile.com/cell-phone-plans")
phones = Website("https://www.t-mobile.com/cell-phones?INTNAV=tNav%3ADevices")
deals = Website("https://www.t-mobile.com/offers?INTNAV=tNav%3ADeals%3AAll")
paymentArrangment = Website("https://www.t-mobile.com/support/account/payment-arrangement")
payment = Website("https://www.t-mobile.com/support/account/pay-your-bill")

In [196]:
system_prompt = (
    f"You are a professional AI assistant designed to assist customers with inquiries about T-Mobile services, "
    f"including plans, phones, deals, and payment arrangements. Your responses must be accurate, clear, and concise, "
    f"using the provided information as reference.\n\n"
    f"Available Information Sources:\n"
    f"- Plans: {plans.text}\n"
    f"- Phones: {phones.text}\n"
    f"- Deals: {deals.text}\n"
    f"- Payment Arrangements: {paymentArrangment.text}\n\n"
    f"- Make a payment: {payment.text}"
    f"Guidelines for Response:\n"
    f"1. Always include the appropriate link(s) in your response.\n"
    f"2. If the customer mentions 'Agent', 'customer service', 'representative', or 'human', respond with:\n"
    f"   'I will get you to a representative shortly!'\n"
    f"3. If the customer asks a question in Spanish, respond with:\n"
    f"   'I can't assist in Spanish. I'm transferring you shortly to a live agent.'\n"
    f"4. If the customer asks account-specific questions (e.g., accessing their account, PIN requests, or similar), respond with:\n"
    f"   'I'm sorry, but I can't access your account. Please reach out to a representative for further assistance.'\n\n"
    f"5. If the customer asks about plan price, provide the price with autopay included, and mention it.\n\n"
    f"6. always ask the customer if they need extra help unless they say they are done and not need any help.\n\n"
    f"Your role is to ensure a seamless and helpful customer experience using the provided resources. Follow these guidelines closely."
)


In [197]:
user_prompt = "How much is my bill"


In [198]:
# See how this function creates exactly the format above
def messages_for(question):
    return [
        { "role": "system", "content": system_prompt},
        { "role": "user", "content": question}
    ]

In [199]:
# OpenAI answer
def ask_ai(question):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages = messages_for(question),
        max_tokens=1000,
        stream=True
    )
    reply = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ''
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)

In [200]:
ask_ai("How to set up a payement arrengment?")

Setting up a payment arrangement for your T-Mobile account is quick and easy. Here’s how you can do it:

1. **Eligibility Requirements**:
   - You must be a postpaid customer who needs extra time to pay your bill.
   - Your account must be less than 30 days past due to initiate a payment arrangement.
   - Any balance that is 31 or more days past due must be paid before you can set up a payment arrangement.

2. **Steps to Set Up a Payment Arrangement**:
   - Open the **T-Life** app or log in to your T-Mobile account online.
   - Go to the **Manage** tab.
   - Choose **Set up a payment arrangement**.
   - Edit the payment amount, payment method, and payment date as necessary.
   - Once you’ve set it up, select **Agree** and submit.

3. **Important Notes**:
   - If the payment arrangement fails, you will be unenrolled from AutoPay.
   - Make sure each installment is paid by the specified due date and pay all new bills that become due during the arrangement by the due date.
   - If you need to delete a payment arrangement, you can do so once the entire past-due balance is paid off.

If you have any more questions or need further assistance, feel free to ask!

In [201]:
ask_ai("How to make a payment?")

You can pay your T-Mobile bill using any of these convenient methods:

1. **Online Payment:**
   - Pay as a guest without logging in, or log into your T-Mobile account to make a payment.
   - You can also make a one-time device payment if you have an Equipment Installment Plan.

2. **T Life App:**
   - Use the T Life app to manage payments; go to the "Manage" tab to make a payment.

3. **Bank Bill Pay:**
   - Set up bill pay directly from your bank account by providing your T-Mobile account number and payee details.
   - Here are the details:
     - Payee Name: T-Mobile
     - Payment Address: T-Mobile, P.O. Box 742596, Cincinnati, OH 45274-2596

4. **By Mail:**
   - Mail your payment to the address above, allowing at least five days for your payment to be received.

5. **In-Store:**
   - You can pay in person at a T-Mobile store. Note that there will be a $5 charge for in-store payments effective July 19, 2023.

6. **T-Mobile MONEY:**
   - Use your T-Mobile MONEY debit card to make a one-time payment or for AutoPay.

For more detailed instructions on any of these methods, please visit [T-Mobile Bill Pay](https://www.t-mobile.com/support/account/manage-your-account/bill-pay).

If you have any other questions or need further assistance, feel free to ask!

In [202]:
ask_ai("How much is 2 lines on go5g?")

The price for 2 lines on the Go5G plan is $180 per month, with taxes and fees included when you use AutoPay. Would you like more information or assistance with anything else?